## <center> ENSF 592 Programming Fundamentals for Data Engineers </center>
<h2> <center> Assignment 7 Bonus: Web data mining (4 points)</center></h2>
<center>
<div class="alert alert-block alert-info">
Due: Wed Nov 20 12am (midnight, Tue to Wed). To be submitted on D2L.
</div></center>




Edit this file and write your solutions to the problems in sections specified with `# Your solution goes here`. Test your code and when you are done, download this notebook as an `.ipynb` file and submit it to D2L. To get this file, in Jupyter notebook you can go to File -> Download as -> Notebook(.ipynb)

## Part 1: Billboard top charts (2 pts)

Complete the functions below to get the best selling albums of a certain week from the [billboard 200](https://www.billboard.com/charts/billboard-200) page. Create a pandas dataframe to store the information. Your dataframe will have (at least) three columns:

* title (the album name)
* artist
* rank

Feel free to extract more information (like the number of weeks that it has been on the chart, a url to the album cover art, ...) if available and you're interested.



In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import re

def get_billboard_top_albums_dataframe(date: str='2001-06-02', count: int=5) -> pd.DataFrame:
    # Your solution goes here
    # Feel free to add auxilary functions, more imports, etc 
    
    req = requests.get('https://www.billboard.com/charts/billboard-200/'+date, verify=False).text

    soup = BeautifulSoup(req, 'lxml')
    
    charts = pd.read_json( soup.find('div', id="charts")['data-charts'])
    return charts[['title','artist_name','rank']].head(count)

    

top_5_albums = get_billboard_top_albums_dataframe(count=5, date='1999-06-02')
top_5_albums

/home/bryce/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


,title,artist_name,rank
0,Millennium,Backstreet Boys,1
1,Ricky Martin,Ricky Martin,2
2,...Baby One More Time,Britney Spears,3
3,Fanmail,TLC,4
4,Star Wars Episode I: The Phantom Menace (Sound...,London Symphony Orchestra (Williams),5


## Music brainz API (2 pts)
Here's a code that connects to the music brainz music database API and searches for the information about the given album. 

In [2]:
import itertools
import unicodedata
import re

_remove_accents = lambda input_str: ''.join((c for c in unicodedata.normalize('NFKD', input_str) if not unicodedata.combining(c)))
_clean_string = lambda s: set(re.sub(r'[^\w\s]', '', _remove_accents(s)).lower().split())
_jaccard = lambda set1, set2: float(len(set1 & set2)) / float(len(set1 | set2))


def search(entity_type:str, query: str):
    return requests.get(
        'http://musicbrainz.org/ws/2/{entity}/'.format(entity=entity_type),
        params={
            'fmt': 'json',
            'query': query
        }
    ).json()


def get_release_url(artist: str, title: str):
    type_ = 'release'
    search_results = search(type_, '%s AND artist:%s' % (title, artist))
    
    artist = _clean_string(artist)
    title = _clean_string(title)

#     print("title = " + str(title) +' artist=' + str(artist))
    for item in search_results.get(type_+'s', []):
        names = list()
        for artists in item['artist-credit']:
            if 'artist' in artists:
                names.append(_clean_string(artists['artist']['name']))
                for alias in artists['artist'].get('aliases', {}):
                    names.append(_clean_string(alias.get('name', '')))
#         print('  title=' + str(_clean_string(item['title'])) + ' names=' + ', '.join(itertools.chain(*names)))

        if _jaccard(_clean_string(item['title']), title) > 0.5 and \
            (any(_jaccard(artist, name) > 0.3 for name in names) or len(names) == 0):
            return 'http://musicbrainz.org/ws/2/{type}/{id}?inc=artist-credits+labels+discids+recordings&fmt=json'.format(id=item['id'], type=type_)
    
    return None



Using the above function, find the track count and disk count of each album (if available) and store it in the pandas data frame. Read the implementation of the `search` function above and see how the `fmt` parameter is passed to the web service in order to make it return the results in `json` format.

The url provided by the `get_release_url` function above, is missing the `inc` parameter. The  `inc` parameter, similar to `fmt`, it can be passed to the server to ask for extra data. Consult the [API docs](https://musicbrainz.org/doc/Development/JSON_Web_Service) to determine the value for `inc` to obtain disk counts of the album (release). The goal is to write another function that takes the url returned by `get_release_url` and supplying the correct values for `fmt` and `inc` in the `requests.get()` `params` argument similar to `search()`

There is other information freely available on the music brainz database. Feel free to explore and scrape more. You can browse the [API docs](https://musicbrainz.org/doc/Development/JSON_Web_Service) for more information. 

In [29]:
# Your solution goes here
top_5_albums = get_billboard_top_albums_dataframe()

for i in range(top_5_albums.shape[0]):
    res = requests.get(get_release_url(top_5_albums.iloc[i]['artist_name'], top_5_albums.iloc[i]['title']),verify=False).json()
    top_5_albums.ix[i, 'track-count'] = res["media"][0]['track-count']
    top_5_albums.ix[i, 'disc-count'] = len(res["media"][0]["discs"])
top_5_albums

/home/bryce/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/bryce/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  
/home/bryce/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  import sys


,title,artist_name,rank,track-count,disc-count
0,Lateralus,Tool,1,13.0,8.0
1,Miss E ...So Addictive,"Missy ""Misdemeanor"" Elliott",2,30.0,1.0
2,Survivor,Destiny's Child,3,19.0,1.0
3,Weezer,Weezer,4,10.0,8.0
4,All For You,Janet,5,3.0,2.0
